In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import cProfile #for checking the nr of calls and execution time
import pstats
from pstats import SortKey

from tqdm.notebook import tqdm
import numpy as np
import multiple_planets_gas_acc as code_gas
from functions_pebble_accretion import *
from functions import *
import functions_plotting as plot
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import pandas as pd
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator, MultipleLocator, AutoMinorLocator
from matplotlib import cm, ticker
from matplotlib import colors
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 



In [ ]:
# disc parameters
params_dict = {'St_const': None, 
               'iceline_radius': None,
                'alpha': 1e-2,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'epsilon_el': 1e-2,
                'epsilon_heat':0.5,
                'v_frag': (1 * u.m/u.s).to(u.au/u.Myr).value,
                'M_dot_gas_star': "star_mass_linear",
                }
params = code_gas.Params(**params_dict, H_r_model='irradiated')

Nsteps = 5000
a_p0 = np.geomspace(1,1, num = 1)
t0 = 0.1 #Myr
t_in = 0.1 #Myr
t_fin = 3.0 #Myr
sigma_gas = sigma_gas_steady_state(a_p0, t0, params)
m0 = M0_pla_Mstar(a_p0, t0, sigma_gas, params)

a_p0 = np.array([a_p0])
m_0 = np.array([m0])
t0 = np.array([t0])

sim_params_dict = {'N_step': Nsteps,
                'm0': m_0,
                'a_p0': a_p0,
                't0': t0,
                't_fin': t_fin,
            }

sim_params = code_gas.SimulationParams(**sim_params_dict)
peb_acc = code_gas.PebbleAccretion(simplified_acc=False)
gas_acc = code_gas.GasAccretion()

In [ ]:
cProfile.run(
    'sim = code_gas.simulate_euler(migration=True, filtering=False,  peb_acc=peb_acc, gas_acc=gas_acc, ' \
    'params=params, sim_params=sim_params, output_folder="sims/gas_acc/test/speed_up")',
    'profile_results.prof'
)

In [ ]:
stats = pstats.Stats('profile_results.prof')
stats.strip_dirs()
stats.sort_stats('call')  # Sort by time
stats.print_stats(100)  

In [ ]:
%prun sim = code_gas.simulate_euler(migration=True, filtering=False, peb_acc=peb_acc, gas_acc=gas_acc, params=params, sim_params=sim_params, output_folder="sims/gas_acc/test/speed_up")
